In [1]:
import os
import random
import numpy as np
from glob import glob
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import torchvision
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import confusion_matrix
import seaborn as sn 
import matplotlib.pyplot as plt

## Seeding

In [2]:
random.seed(42)
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

## Dataset

In [3]:
dataset_path = "/media/nikhil/New Volume/ML_DATASET/Beans/"

train_ratio = 0.70
valid_ratio = 0.15
test_ratio = 0.15

In [4]:
class_folders = sorted(os.listdir(dataset_path))

In [5]:
train_x, valid_x, test_x = [], [], []
train_y, valid_y, test_y = [], [], []

In [6]:
# Iterate through each class folder
for i, class_folder in enumerate(class_folders):
    class_path = os.path.join(dataset_path, class_folder)
    # images = os.listdir(class_path)
    images = sorted(glob(os.path.join(class_path, "*")))
    num_images = len(images)
    
    # Calculate number of samples for each split
    num_train = int(train_ratio * num_images)
    num_valid = int(valid_ratio * num_images)
    num_test = num_images - num_train - num_valid
    
    # Allocate samples to each split
    random.shuffle(images)
    train_samples = images[:num_train]
    valid_samples = images[num_train:num_train + num_valid]
    test_samples = images[num_train + num_valid:]
    
    train_x += train_samples
    valid_x += valid_samples
    test_x += test_samples
    
    train_y += [i] * len(train_samples)
    valid_y += [i] * len(valid_samples)
    test_y += [i] * len(test_samples)

In [7]:
print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")

Train: 8607/8607 - Valid: 1838/1838 - Test: 1864/1864


## Dataset Loader

In [8]:
test_transform = transforms.Compose([
    # transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(image).convert('RGB')
        label = np.array(label)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [10]:
test_dataset = CustomDataset(test_x, test_y, transform=test_transform)

batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Testing

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, dim_feedforward=128, dropout=0.1):
        super().__init__()

        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        ## Reshape
        src = x.view((x.shape[0], x.shape[1], x.shape[2]*x.shape[3]))

        # Self-attention layer
        src2 = self.attn(src, src, src, attn_mask=mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        # Feedforward layer
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)

        src = src.view((x.shape[0], x.shape[1], x.shape[2], x.shape[3]))

        return src

In [13]:
model = torchvision.models.squeezenet1_0(pretrained=False)
model.classifier._modules["1"] = TransformerBlock(169, 13)
model.classifier.append(nn.Conv2d(512, 20, kernel_size=(1, 1)))
model.num_classes = 20
    
model.load_state_dict(torch.load('squeezenet1_0_best_weights.pth'))  # Load the trained weights
model.to(device)
model.eval()  # Set the model to evaluation mode

/home/nikhil/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nikhil/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [14]:
true_labels = []
predicted_labels = []

In [ ]:
# Iterate through the test dataset and make predictions
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [ ]:
# Calculate accuracy, recall, precision, F1 score, and F2 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='macro')
precision = precision_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
f2 = fbeta_score(true_labels, predicted_labels, beta=2, average='macro')

print(f"Test Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"F2 Score: {f2}")

## Confusion Matrix

In [ ]:
true_labels = np.array(true_labels, dtype=np.int32)
predicted_labels = np.array(predicted_labels, dtype=np.int32)

In [ ]:
labels = range(0, len(class_folders), 1)

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels, labels=labels)

In [ ]:
list(cm.tolist())

In [ ]:
plt.figure(figsize=(20, 10))
hm = sn.heatmap(data=cm, annot=True, fmt='d')
plt.savefig("heatmap.png")

## Curve Line

In [ ]:
df = pd.read_csv("training_val_losses.csv")

In [ ]:
plt.figure(figsize=(10, 6))
# sn.lineplot(data=df[['Train Loss', 'Val Loss']])
plt.plot(df['Train Loss'], label='Train Loss')
plt.plot(df['Val Loss'], label='Validation Loss')
plt.title('Train Loss vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(title='', loc='upper right')

plt.savefig("loss_curve.png")